# Processing netcdf files

load required R packages

In [36]:
install.packages("ncdf4")
install.packages("raster")
install.packages("rgdal")
install.packages("leaflet")

require(dplyr)
require(ncdf4)
require(ggplot2)
require(raster)
require(leaflet)
require(rgdal)

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Warning message in install.packages("rgdal"):
“installation of package ‘rgdal’ had non-zero exit status”Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Loading required package: rgdal
Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘rgdal’”

Source function for aggregating netcdf

In [37]:
source("read_nc.R")

Function call:<br>
<p>read_nc(file,stat="mean", timesteps=c(1:364), varno=3, unitfactor=1, cleanNA=T)<p>
                 
Parameters:<br>
* file: netcdf file
* stat: "mean" or "90pc". Default = "mean"
* timesteps: time range over which we should aggregate. Default = c(1:364)
* varno: the number of the variable we are interested in. Default=3
* unitfactor: conversion factor. Default=1
               e.g.  use unitfactor=0.032*0.7 for converting Dissolved Oxygen mmol/m3 -> mg/L -> ml/L
* cleanNA: drop data points where the variable is NA. Default = TRUE
               

In [38]:
# define function parameters for Chl

ncfile="../../../shared/martini/martini800_v2_Chl_surf.nc"
statistic="90pc"
timesteps=c(1:364)
varno=5
unitfactor=1

# call the function
df <- read_nc(ncfile, statistic, timesteps, varno, unitfactor)



[1] "lon_rho"   "lat_rho"   "Cs_r"      "h"         "light_Chl"
[1] "light_Chl[mg/m^3] [ndims=4] averaged light/total chlorophyll a"


In [39]:
names(df)

[1] "id"    "lat"   "lon"   "value" "err1"  "err2"

In [40]:
# read table matching id from nc data to positions in the epsg3035 grid
df_points<-read.table(file="grid_epsg3035.txt",header=T,stringsAsFactors=F,sep=";")

# rename the 'value' parameter in df
df <- df %>%
    dplyr::select(ID=id,value)

param = "Chl"
names(df)[names(df)=="value"]<-param

# join the dataframe to positions in EPSG3035
df <- df_points %>%
  left_join(df,by="ID")


In [41]:
# load the 'blank' raster file
r <- raster("blank_raster.tif")
crs(r) <- CRS('+init=epsg:3035')

x <- rasterize(df[, 1:2], r, df[,4], fun=mean)
  
# save the raster file
  #rf <- writeRaster(x, filename=paste0("raster/",param,".grd",overwrite=TRUE))
  
plot(x, main=param)


ERROR: Error in .rasterObjectFromFile(x, band = band, objecttype = "RasterLayer", : Cannot create RasterLayer object from this file; perhaps you need to install rgdal first
